In [1]:
from pulp import * 
import pandas as pd
import numpy as np
#===========
n_warehouses = 2
n_customers = 4
# Cost Matrix
cost_matrix = np.array([[1, 3, 0.5, 4],
[2.5, 5, 1.5, 2.5]])
# Demand Matrix
cust_demands = np.array([35000, 22000, 18000, 30000])
# Supply Matrix
warehouse_supply = np.array([60000, 80000])
#===========
model = LpProblem("Supply-Demand-Problem", LpMinimize)
#===========
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1, n_warehouses+1)]
variable_names.sort()
print("Variable Indices:", variable_names)
#===========
DV_variables = LpVariable.matrix("X", variable_names, cat = "Integer", lowBound= 0 )
allocation = np.array(DV_variables).reshape(2,4)
print("Decision Variable/Allocation Matrix: ")
print(allocation)
#===========
obj_func = lpSum(allocation*cost_matrix)
print(obj_func)
model += obj_func
print(model)
#===========
#Supply Constraints
for i in range(n_warehouses):
    # start: for(n_warehouses)
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)
    #===========
    # Demand Constraints
    for j in range(n_customers):
        print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
        model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)
        #===========
        model.writeLP("Supply_demand_prob.lp")
        #===========
        #model.solve()
        model.solve(PULP_CBC_CMD())
        status = LpStatus[model.status]
        print(status)
        #===========
        print("Total Cost:", model.objective.value())
        # Decision Variables
        for v in model.variables():
            try:
                print(v.name,"=", v.value())
            except:
                print("error couldnt find value")
    # end: for(n_warehouses)
# Warehouse 1 and Warehouse 2 required capacity
for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Variable Indices: ['11', '12', '13', '14', '21', '22', '23', '24']
Decision Variable/Allocation Matrix: 
[[X_11 X_12 X_13 X_14]
 [X_21 X_22 X_23 X_24]]
X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24
Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

X_11 + X_12 + X_13 + X_14 <= 60000
X_11 + X_21 >= 35000
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/lugas/Python/prak_Optimasi/myenv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/5238d9269b084b1aa6234137ba0b0e12-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/5238d9269b084b1aa6234137ba0b0e12-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At 

PulpError: overlapping constraint names: Demand_Constraints_0